In [8]:
import hashlib
import json
import rsa

# Generate public-private keys for Manufacturer
manufacturer_keys = rsa.newkeys(512)
manufacturer_public_key = manufacturer_keys[0]  # Public key for verification
manufacturer_private_key = manufacturer_keys[1]  # Private key for signing

class Block:
    def __init__(self, index, medicine_data, prev_hash, entity):
        self.index = index
        self.medicine_data = medicine_data  # Medicine details
        self.prev_hash = prev_hash
        self.hash = self.calculate_hash()  # Block hash for structure integrity
        self.entity = entity
        self.signature = None  # Signature from Manufacturer for medicine details

    def calculate_hash(self):
        """Create a hash of the block (excluding the signature)."""
        block_content = json.dumps(self.medicine_data, sort_keys=True) + self.prev_hash
        return hashlib.sha256(block_content.encode()).hexdigest()

    def sign_medicine_data(self):
        """Create a digital signature using the Manufacturer's private key for the medicine data only."""
        medicine_data_hash = hashlib.sha256(json.dumps(self.medicine_data, sort_keys=True).encode()).hexdigest()
        self.signature = rsa.sign(medicine_data_hash.encode(), manufacturer_private_key, 'SHA-256')

    def verify_signature(self):
        """Verify the Manufacturer's digital signature on the medicine data."""
        try:
            medicine_data_hash = hashlib.sha256(json.dumps(self.medicine_data, sort_keys=True).encode()).hexdigest()
            rsa.verify(medicine_data_hash.encode(), self.signature, manufacturer_public_key)
            return True  # If no exception, the signature is valid
        except rsa.VerificationError:
            return False  # Signature is invalid

# Blockchain list
blockchain = []
fraud_detected = False  # Flag to track if fraud has been detected

def add_block(medicine_data, entity):
    global fraud_detected

    if fraud_detected:
        print(f"❌ Fraud detected earlier! No more blocks will be added.\n")
        return  # Stop adding blocks if fraud has been detected earlier

    prev_hash = blockchain[-1].hash if blockchain else "000000"  # Genesis block case

    # Create a new block
    new_block = Block(len(blockchain) + 1, medicine_data, prev_hash, entity)

    # Manufacturer adds first block (Genesis block)
    if entity == "Manufacturer":
        new_block.sign_medicine_data()  # Manufacturer signs the medicine data only
        blockchain.append(new_block)
        print(f"\n✅ Block {new_block.index} added successfully by {entity}.")
        print(f"🔹 Manufacturer's Block Data: {new_block.medicine_data}\n")
        return

    # For other entities, verify the received block from the previous stage
    if blockchain:
        received_block = blockchain[-1]  # Get the last block added

        # Print received details
        print(f"\n🔹 {entity} received block from previous entity:")
        print(f"   ➤ Data: {received_block.medicine_data}")

        # Verify the Manufacturer's signature on the medicine data using the public key
        if received_block.verify_signature():
            print(f"✅ No fraud at {received_block.entity} level.")
        else:
            print(f"❌ Fraud detected at Manufacturer level! Block rejected.\n")
            fraud_detected = True  # Set the fraud flag to True
            return

        # Check if data is modified at this stage
        if received_block.medicine_data != medicine_data:
            print(f"❌ Fraud detected at {entity} level! Data modified. Block rejected.\n")
            fraud_detected = True  # Set the fraud flag to True
            return
        
        # If everything is valid, add the new block
        new_block.sign_medicine_data()  # Sign the new block for the next entity
        blockchain.append(new_block)
        print(f"✅ Block {new_block.index} added successfully by {entity}.\n")


def get_input():
    """Function to get user input for adding a block."""
    global fraud_detected

    if fraud_detected:
        print("❌ Fraud detected earlier. No further input will be processed.")
        return

    # Get the entity input (e.g., Manufacturer, Distributor, Wholesaler, etc.)
    entity = input("Enter entity (Manufacturer, Distributor, Wholesaler, Retailer): ").capitalize()

    if entity not in ["Manufacturer", "Distributor", "Wholesaler", "Retailer"]:
        print("❌ Invalid entity. Please enter a valid entity (Manufacturer, Distributor, Wholesaler, Retailer).")
        return

    # Get the medicine details input (e.g., Batch, Manufacturer, Expiry)
    batch = input("Enter batch number: ")
    manufacturer = input("Enter manufacturer name: ")
    expiry = input("Enter expiry date (e.g., 2025): ")

    # Prepare medicine data
    medicine_data = {"Batch": batch, "Manufacturer": manufacturer, "Expiry": expiry}

    # Add block with provided data and entity
    add_block(medicine_data, entity)


# Start with the Manufacturer adding the first block
get_input()

# Other entities (Distributor, Wholesaler, Retailer) will add blocks based on user i nput
while not fraud_detected:
    get_input()

Enter entity (Manufacturer, Distributor, Wholesaler, Retailer):  Manufacturer
Enter batch number:  123
Enter manufacturer name:  apollo
Enter expiry date (e.g., 2025):  2025



✅ Block 1 added successfully by Manufacturer.
🔹 Manufacturer's Block Data: {'Batch': '123', 'Manufacturer': 'apollo', 'Expiry': '2025'}



Enter entity (Manufacturer, Distributor, Wholesaler, Retailer):  Distributor
Enter batch number:  123
Enter manufacturer name:  apollo
Enter expiry date (e.g., 2025):  2025



🔹 Distributor received block from previous entity:
   ➤ Data: {'Batch': '123', 'Manufacturer': 'apollo', 'Expiry': '2025'}
✅ No fraud at Manufacturer level.
✅ Block 2 added successfully by Distributor.



Enter entity (Manufacturer, Distributor, Wholesaler, Retailer):  Wholesaler
Enter batch number:  1234
Enter manufacturer name:  apollo
Enter expiry date (e.g., 2025):  2025



🔹 Wholesaler received block from previous entity:
   ➤ Data: {'Batch': '123', 'Manufacturer': 'apollo', 'Expiry': '2025'}
✅ No fraud at Distributor level.
❌ Fraud detected at Wholesaler level! Data modified. Block rejected.

